In [20]:
import pandas as pd

# Read the CSV file into a DataFrame
player_bios = pd.read_csv('player_bio.csv')
player_bios.rename(columns={player_bios.columns[0]: 'Player_Name'}, inplace=True)

# You can now work with the data using the DataFrame
player_bios

,Player_Name,Team,Number,Position,Birthdate,College,Draft_Year,Draft_Round,Draft_Pick_Overall,Draft_Team,Height,Weight,Age
0,Ben DiNucci,Broncos,#11,QB,11/24/1996,James Madison,2020,7,231,DAL,6' 2,215 lbs,26
1,Jarrett Stidham,Broncos,#4,QB,8/8/1996,Auburn,2019,4,133,NE,6' 3,215 lbs,27
2,Russell Wilson,Broncos,#3,QB,11/29/1988,Wisconsin,2012,3,75,SEA,5' 11,215 lbs,34
3,Tyler Badie,Broncos,#36,RB,2/7/2000,Missouri,2022,6,196,BAL,5' 8,197 lbs,23
4,Jaleel McLaughlin,Broncos,#38,RB,9/13/2000,Youngstown St,undrafted,undrafted,undrafted,undrafted,5' 7,187 lbs,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Caden Sterns,Broncos,#30,S,11/2/1999,Texas,2021,5,152,DEN,6' 1,207 lbs,23
74,Delarrin Turner-Yell,Broncos,#32,S,12/16/1999,Oklahoma,2022,5,152,DEN,5' 11,200 lbs,23
75,Wil Lutz,Broncos,#16,PK,7/7/1994,Georgia State,undrafted,undrafted,undrafted,undrafted,5' 11,184 lbs,29
76,Riley Dixon,Broncos,#9,P,8/24/1993,Syracuse,2016,7,228,DEN,6' 4,221 lbs,30


## Fixing Players without Numbers (to be moved to scarper)

In [25]:
# Iterate through the DataFrame and update the "Position" and "Number" columns
for index, row in player_bios.iterrows():
    if not row['Number'].startswith('#'):
        print(player_bios.at[index, 'Player_Name'], " does not have")
        player_bios.at[index, 'Position'] = player_bios.at[index, 'Number']
        player_bios.at[index, 'Number'] = None

Keondre Coburn  does not have
             Player_Name     Team Number Position   Birthdate        College  \
0            Ben DiNucci  Broncos    #11       QB  11/24/1996  James Madison   
1        Jarrett Stidham  Broncos     #4       QB    8/8/1996         Auburn   
2         Russell Wilson  Broncos     #3       QB  11/29/1988      Wisconsin   
3            Tyler Badie  Broncos    #36       RB    2/7/2000       Missouri   
4      Jaleel McLaughlin  Broncos    #38       RB   9/13/2000  Youngstown St   
..                   ...      ...    ...      ...         ...            ...   
73          Caden Sterns  Broncos    #30        S   11/2/1999          Texas   
74  Delarrin Turner-Yell  Broncos    #32        S  12/16/1999       Oklahoma   
75              Wil Lutz  Broncos    #16       PK    7/7/1994  Georgia State   
76           Riley Dixon  Broncos     #9        P   8/24/1993       Syracuse   
77      Mitchell Fraboni  Broncos    #48       LS  10/28/1996  Arizona State   

   Draft_

In [26]:
contain_values = player_bios[player_bios['Player_Name'].str.contains('Keondre Coburn')]
print (contain_values)

       Player_Name     Team Number Position  Birthdate College Draft_Year  \
40  Keondre Coburn  Broncos   None       DT  5/23/2000   Texas       2023   

   Draft_Round Draft_Pick_Overall Draft_Team Height   Weight  Age  
40           6                194         KC   6' 2  332 lbs   23  


In [7]:
import os
import glob
import pandas as pd

# Define the root directory where you want to search for sub-folders and CSV files
root_directory = "PlayerData"  # Assuming "PlayerData" is in your current working directory

# Define the path to the player_bios file
player_bios_file = "player_bios.csv"  # Replace with the actual path

# Read the player_bios file into a DataFrame
player_bios_df = pd.read_csv(player_bios_file)

# Create dictionaries to map positions to units
offense_positions = {"QB", "RB", "FB", "WR", "TE", "C", "G", "OT"}
defense_positions = {"DE", "DT", "LB", "CB", "S"}
special_teams_positions = {"PK", "P", "LS"}

# Initialize dictionaries to store merged dataframes for each unit
merged_dataframes = {
    "Offense": pd.DataFrame(),
    "Defense": pd.DataFrame(),
    "Special_Teams": pd.DataFrame(),
}

# Get a list of all sub-folders
sub_folders = [f.path for f in os.scandir(root_directory) if f.is_dir()]

# Iterate through the sub-folders and search for CSV files
for sub_folder in sub_folders:
    category = os.path.basename(sub_folder)  # Extract the category from the folder name
    
    # Define the pattern to search for CSV files (e.g., all files with a .csv extension)
    csv_pattern = os.path.join(sub_folder, "*.csv")
    
    # Use glob to find CSV files matching the pattern
    csv_files = glob.glob(csv_pattern)
    
    # Iterate through the CSV files in each category
    for csv_file in csv_files:
        player_name = os.path.basename(sub_folder)  # Extract the player name from the folder name
        
        # Use player_name to look up the player's position from player_bios
        player_info = player_bios_df[player_bios_df["Player Name"] == player_name]
        if not player_info.empty:
            player_position = player_info.iloc[0]["Position"]
            
            # Determine the unit (Offense, Defense, Special_Teams) based on the position
            if player_position in offense_positions:
                unit = "Offense"
            elif player_position in defense_positions:
                unit = "Defense"
            elif player_position in special_teams_positions:
                unit = "Special_Teams"
            else:
                unit = "Unknown"  # Handle positions not in the mapping
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(csv_file)
            
            # Merge the DataFrame into the appropriate unit
            merged_dataframes[unit] = pd.concat([merged_dataframes[unit], df], ignore_index=True)
        else:
            print(f"Player '{player_name}' not found in player_bios.")
            
# Now, the 'merged_dataframes' dictionary contains DataFrames merged by unit (e.g., "Offense", "Defense", "Special_Teams").


FileNotFoundError: [Errno 2] No such file or directory: 'player_bios.csv'

In [11]:
import os
import glob
import pandas as pd

# Define the root directory where you want to search for sub-folders and CSV files
root_directory = "PlayerData"  # Assuming "PlayerData" is in your current working directory

# Get a list of all sub-folders
sub_folders = [f.path for f in os.scandir(root_directory) if f.is_dir()]

# Initialize a dictionary to store merged dataframes
merged_dataframes = {}

# Iterate through the sub-folders and search for CSV files
for sub_folder in sub_folders:
    # Define the pattern to search for CSV files (e.g., all files with a .csv extension)
    csv_pattern = os.path.join(sub_folder, "*.csv")
    
    # Use glob to find CSV files matching the pattern
    csv_files = glob.glob(csv_pattern)
    
    # Iterate through the CSV files and merge them by filename
    for csv_file in csv_files:
        file_name = os.path.splitext(os.path.basename(csv_file))[0]  # Extract the filename without extension
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file)
        
        # Check if a DataFrame with the same name already exists in the dictionary
        if file_name in merged_dataframes:
            # Append the data to the existing DataFrame
            merged_dataframes[file_name] = pd.concat([merged_dataframes[file_name], df], ignore_index=True)
        else:
            # Create a new entry in the dictionary for the DataFrame
            merged_dataframes[file_name] = df

# Now, the 'merged_dataframes' dictionary contains DataFrames merged by filename.
# You can access each merged DataFrame using the filename as the key.


C:\Users\Marc\AppData\Local\Temp\ipykernel_6872\552861867.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_dataframes[file_name] = pd.concat([merged_dataframes[file_name], df], ignore_index=True)
C:\Users\Marc\AppData\Local\Temp\ipykernel_6872\552861867.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_dataframes[file_name] = pd.concat([merged_dataframes[file_name], df], ignore_index=True)
C:\Users\Marc\AppData\Local\Temp\ipykernel_6872\552861867.py:32: FutureWarning: The be

In [10]:
merged_dataframes['Opponent'].head()

,Opponent,REC,YDS,AVG,TD,LNG,CAR,YDS.1,AVG.1,TD.1,...,RTG,1-19,20-29,30-39,40-49,50+,FG,FG%,XP,PTS
0,vs MIA,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,vs OAK,5.0,34.0,6.8,0.0,15,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,vs WAS,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,vs OAK,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vs WAS,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
